In [1]:
import numpy as np
from scipy import signal
from keras.datasets import mnist
from PIL import Image

2023-12-08 16:42:29.023592: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### CNN Layer

In [21]:
class CNNLayer:
    def __init__(self, kernel_size, number_of_kernels, input_shape):
        assert len(input_shape) == 3, "Input shape must be 3D"
        self.input_channels = input_shape[2]
        
        self.input_shape = input_shape
        self.kernel_size = kernel_size
        self.number_of_kernels = number_of_kernels
        self.kernels = np.random.randn(kernel_size, kernel_size, self.input_channels, number_of_kernels)
        
        # output shape
        self.output_shape = (input_shape[0] - kernel_size + 1, input_shape[1] - kernel_size + 1, number_of_kernels)
        self.biases = np.random.randn(*self.output_shape)
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct"

        output = np.zeros(self.output_shape)
        for i in range(self.number_of_kernels):
            for j in range(self.input_channels):
                output[:, :, i] += signal.correlate2d(input[:, :, j], self.kernels[:, :, j, i], mode='valid')
        
        output += self.biases
        
        return output
    
    def backward(self, input, gradWRTMyOutput, learning_rate):
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at CNN layer with kernel size: " + str(self.kernel_size)
        assert input.shape == self.input_shape, "Input shape is not correct at CNN layer with kernel size: " + str(self.kernel_size)
        
        # Compute gradient with respect to kernels
        kernel_grads = np.zeros(self.kernels.shape)
        for i in range(self.number_of_kernels):
            for j in range(self.input_channels):
                kernel_grads[:, :, j, i] = signal.correlate2d(input[:, :, j], gradWRTMyOutput[:, :, i], mode='valid')
        
        # Compute gradient with respect to biases
        bias_grads = gradWRTMyOutput.copy()
        
        # Compute gradient with respect to input
        input_grads = np.zeros(input.shape)
        for j in range(self.input_channels):
            for i in range(self.number_of_kernels):
                input_grads[:, :, j] += signal.convolve2d(gradWRTMyOutput[:, :, i], self.kernels[:, :, j, i], mode='full')
        
        # Update kernels and biases
        self.kernels -= learning_rate * kernel_grads
        self.biases -= learning_rate * bias_grads
        
        return input_grads

#### Sigmoid Layer
Input: (x, y, z) where z is the number of channels

Output: (x', y', z')

Error_Grad_WRT_Input: Error_Grad_WRT_Output * Output_Grad_WRT_Input , where * is element-wise multiplication

In [16]:
class SigmoidLayer:
    def __init__(self, input_shape):
        # Input shape is 3D
        assert len(input_shape) == 3, "Input shape must be 3D"
        self.input_shape = input_shape
        self.output_shape = input_shape
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct"
        self.output = 1.0 / (1.0 + np.exp(-input))
        return self.output
    
    def sigmoid_derivative(self, x):
        return x * (1.0 - x)
    
    def backward(self, input, gradWRTMyOutput):
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at sigmoid layer"
        assert input.shape == self.input_shape, "Input shape is not correct at sigmoid layer"
        # element-wise gradient
        gradMyOutputWRTMyInput = self.sigmoid_derivative(self.output)
        # element-wise multiplication
        return gradWRTMyOutput * gradMyOutputWRTMyInput

#### Average Pooling Layer
Input: (x, y, z) where z is the number of channels

Output: (x', y', z)

Error Gradient

$\frac{dE}{dX_{ijk}} = \sum_{m,n} \frac{dE}{dY_{mnk}} \times \frac{1}{\text{pool\_size}^2} \times \mathbb{\theta}_{i, j}$ where $\mathbb{\theta}_{i, j}$ is 1 if $X_{ijk}$ was included when calculating $Y_{mnk}$ and 0 otherwise.

In [25]:
class AveragePoolLayer:
    def __init__(self, input_shape, pool_size, stride):
        # Input shape is 3D
        assert len(input_shape) == 3, "Input shape must be 3D"
        # input shape must be divisible by pool size
        assert input_shape[0] % pool_size == 0, "Input shape must be divisible by pool size"
        self.input_shape = input_shape
        self.output_shape = (input_shape[0] // stride, input_shape[1] // stride, input_shape[2])
        self.pool_size = pool_size
        self.stride = stride
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct"
        self.output = np.zeros(self.output_shape)
        for i in range(self.output_shape[0]):
            for j in range(self.output_shape[1]):
                for k in range(self.output_shape[2]):
                    self.output[i, j, k] = np.mean(input[i*self.stride:i*self.stride+self.pool_size, j*self.stride:j*self.stride+self.pool_size, k])
        return self.output

    def backward(self, input, gradWRTMyOutput):
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at average pool layer"
        assert input.shape == self.input_shape, "Input shape is not correct at average pool layer"
        
        # Compute gradient with respect to input
        avg_pool_n = self.pool_size * self.pool_size # number of elements involved in the average pool operation
        input_grads = np.zeros(input.shape)
        
        for channel in range(self.input_shape[2]):
            for i in range(self.output_shape[0]):
                for j in range(self.output_shape[1]):
                    # Only loop over the elements involved in the average pool operation
                    for x in range(i*self.stride, i*self.stride+self.pool_size):
                        for y in range(j*self.stride, j*self.stride+self.pool_size):
                            input_grads[x, y, channel] += gradWRTMyOutput[i, j, channel] / avg_pool_n
        
        return input_grads

#### Flatten Layer

In [27]:
class FlattenLayer():
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.output_shape = (np.prod(input_shape),)
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct"
        self.output = input.flatten()
        return self.output
    
    def backward(self, input, gradWRTMyOutput):
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at flatten layer"
        assert input.shape == self.input_shape, "Input shape is not correct at flatten layer"
        return gradWRTMyOutput.reshape(self.input_shape)

#### Softmax Layer
Softmax Derivative: $\frac{dS_i}{dX_j} = S_i(1 - S_i)$ if $i = j$ and $-S_iS_j$ otherwise

In matrix form, obtain $C = softmax(x) \times softmax(x)^T$ and $D = diag(softmax(x))$ where $D$ is a diagonal matrix with the softmax values on the diagonal and 0 elsewhere.

$K = D - C$
Then, $\frac{dS}{dX} = K \times \frac{dE}{dO}$ where $O$ is the output of the softmax layer and $E$ is the error.

In [31]:
class SoftmaxLayer:
    def __init__(self, input_shape):
        # Input shape is 1D
        assert len(input_shape) == 1, "Input shape must be 1D"
        self.input_shape = input_shape
        self.output_shape = input_shape
    
    def softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x))
    
    def forward(self, input):
        assert input.shape == self.input_shape, "Input shape is not correct"
        self.output = np.exp(input) / np.sum(np.exp(input))
        return self.output
    
    def backward(self, input, gradWRTMyOutput):
        assert gradWRTMyOutput.shape == self.output_shape, "Grad shape is not correct at softmax layer"
        assert input.shape == self.input_shape, "Input shape is not correct at softmax layer"
        
        # Compute gradient with respect to input
        input_grads = np.zeros(input.shape)
        inp_softmax = self.softmax(input)
        softmax_matrix = np.reshape(inp_softmax, (inp_softmax.shape[0], 1)) @ np.reshape(inp_softmax, (1, inp_softmax.shape[0]))
        softmax_matrix = -1 * softmax_matrix
        np.fill_diagonal(softmax_matrix, inp_softmax * (1 - inp_softmax))
        input_grads = softmax_matrix @ gradWRTMyOutput
        
        return input_grads

In [8]:
# Load the MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Display first image
Image.fromarray(x_train[0])

In [9]:
# Normalize the data
x_train = x_train / 255
x_test = x_test / 255

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) # (height, width, channel)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1) # (height, width, channel)

# One-hot encode the labels
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]